# Required Packages Installation

In [ ]:
!pip install openai
!pip install gradio

# Import Required Libraries and Setup OpenAI API Key

In [ ]:
# Import required libraries
import os  # for handling environment variables
import openai  # for interacting with the OpenAI API
import gradio as gr  # for building a simple GUI in Python
from google.colab import userdata  # Colab-specific module to manage user data

# Set OpenAI API key
# Retrieves the OPENAI_API_KEY stored in Google Colab’s userdata and sets it as an environment variable.
# Note: This will only work in Google Colab where the userdata module is available
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Helper Functions

Get Completion from OpenAI

In [ ]:
client = openai.OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    """
    Calls the OpenAI API to generate a response based on provided messages.

    Parameters:
    - messages (list): A list of dictionaries containing the role ('system', 'user', 'assistant') and content of each message in the conversation.
    - model (str): The name of the model to use. Default is "gpt-3.5-turbo".
    - temperature (float): Controls the randomness of the response. Lower values make the response more focused and deterministic.

    Returns:
    - str: The assistant's response text from the OpenAI API.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        # Extract and return the assistant's message content
        return response.choices[0].message.content
    except openai.error.OpenAIError as e:
        print(f"API Error: {e}")
        # Return an error message if the API call fails
        return "Sorry, I encountered an error while processing your request."

Function to collect user messages and display assistant responses (GUI)

In [ ]:
def chatbot(user_input):
    global context  # Access global context
    # Add user's message to context
    context.append({'role': 'user', 'content': user_input})

    # Check if user indicates end of conversation
    if user_input.lower() in ["end order", "finalize order", "thank you"]:
        # Generate and return summary
        summary = generate_order_summary()
        context.clear()  # Clear context for a new conversation
        return "Order Summary:\n" + summary

    # Get response from OpenAI
    response = get_completion_from_messages(context)
    # Append assistant's response to context
    context.append({'role': 'assistant', 'content': response})
    return response

Generate Order Summary in JSON format

In [ ]:
def generate_order_summary():
    """
    Summarizes the user's order in JSON format, itemizing the price for each category (burgers, sides, drinks, dessert) and providing the total price.

    Returns:
    - str: A JSON summary of the order as generated by the OpenAI API.
    """
    # Copy context to avoid modifying the original conversation
    messages = context.copy()
    # Append a system message to instruct the assistant to create a JSON order summary
    messages.append(
        {'role': 'system', 'content': 'Create a JSON summary of the previous food order. \
         Itemize the price for each item with the following fields: \
         1) burgers, 2) sides, 3) drinks, 4) dessert, 5) total price.'}
    )
    # Call OpenAI API to get the summary and return it
    return get_completion_from_messages(messages)

# Panel GUI Initialization

In [ ]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a burger restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
If the customer asking for the item outside our menu, \
response with displaying our menu to give the customer reference. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
You respond in a short, very conversational friendly style. \
The menu includes \
Cheeseburger $6 \
Double burger $8 \
Chicken burger $6 \
Fries $3 \
Onion rings $3 \
Chicken nuggets $4 \
Coke $1 \
Iced Tea $1 \
Milkshake $3 \
Ice Cream $3
"""} ]  # Initial system message to set the context of the assistant


# Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🍔 OrderBot Chatbot")
    chat_input = gr.Textbox(placeholder="Enter your message here...", label="User Input")
    chat_output = gr.Textbox(label="Chatbot Response", lines=10, interactive=False)  # Larger output box

    def submit(user_input):
        bot_response = chatbot(user_input)
        return bot_response, ""

    # Submit button clears input after submission and updates the output
    chat_input.submit(submit, chat_input, [chat_output, chat_input])

# Launch the Gradio app
demo.launch()

# Review

Code Reviewed by ChatGPT: https://chatgpt.com/share/671ee927-bc0c-800b-b667-7fa8d700d1dd